In [6]:
import pandas as pd
river_file_path = './River.xlsx'

river_df = pd.ExcelFile(river_file_path).parse(header=1)

mapping = {
    '탄천_자동': '송파',
    '중랑천_자동': '동대문',
    '선유_자동': '영등포',
    '안양천_자동': '구로',
    '노량진_자동': '현충원'
}


river_df['측정소명'] = river_df['측정소명'].replace(mapping)



updated_river_path = '/Users/jusuung/Desktop/주수웅/Updated_River_with_Mapping.xlsx'
river_df.to_excel(updated_river_path, index=False)



In [7]:
import pandas as pd


temp_file_path = './Temp.xlsx'
updated_river_file_path = './Updated_River_with_Mapping.xlsx'

temp_df = pd.ExcelFile(temp_file_path).parse()
updated_river_df = pd.ExcelFile(updated_river_file_path).parse()

# 빈칸인 값들을 0으로 채움 (NaN(문자열)으로 인식하기 때문에)
temp_df = temp_df.fillna(0)
updated_river_df = updated_river_df.fillna(0)

# '측정소명'을 '지점명'으로 변경
updated_river_df = updated_river_df.rename(columns={'측정소명': '지점명'})

# 시간 데이터 정리
temp_df['일시'] = pd.to_datetime(temp_df['일시'])
updated_river_df['시간'] = pd.to_datetime(updated_river_df['시간'], format='%H:%M:%S', errors='coerce')
updated_river_df['일시'] = pd.to_datetime(updated_river_df['날짜'].astype(str) + ' ' + updated_river_df['시간'].dt.time.astype(str))

# '지점명'과 '일시' 기준으로 병합
merged_df = pd.merge(temp_df, updated_river_df, how='inner', on=['지점명', '일시'])

# 결과를 시간 순으로 정렬 데이터 간 시간순이 역행 순행으로 되어있기 떄문
merged_df = merged_df.sort_values(by='일시')

#시간만 표시 보기편하게 해줌 

if '시간' in merged_df.columns:
    merged_df['시간'] = pd.to_datetime(merged_df['시간'], errors='coerce').dt.strftime('%H:%M:%S')

# 결과 저장
final_merged_path = '/Users/jusuung/Desktop/주수웅/Final_Merged_Time_Corrected.xlsx'
merged_df.to_excel(final_merged_path, index=False)

print(f"최종 파일이 저장되었습니다: {final_merged_path}")


최종 파일이 저장되었습니다: /Users/jusuung/Desktop/주수웅/Final_Merged_Time_Corrected.xlsx


In [8]:
import pandas as pd

# 병합된 파일 불러오기
merged_file_path = './Final_Merged_Time_Corrected.xlsx'
merged_df = pd.read_excel(merged_file_path)

# 지점별로 데이터 나누기
unique_stations = merged_df['지점'].unique()

for station in unique_stations:
    station_df = merged_df[merged_df['지점'] == station]
    output_path = f'./Merged_Station_{station}.xlsx'
    station_df.to_excel(output_path, index=False)
    print(f"{station}번 지점 데이터가 저장되었습니다: {output_path}")


403번 지점 데이터가 저장되었습니다: ./Merged_Station_403.xlsx
510번 지점 데이터가 저장되었습니다: ./Merged_Station_510.xlsx
889번 지점 데이터가 저장되었습니다: ./Merged_Station_889.xlsx
408번 지점 데이터가 저장되었습니다: ./Merged_Station_408.xlsx
423번 지점 데이터가 저장되었습니다: ./Merged_Station_423.xlsx
